In [1]:
from nba_api.stats.library.parameters import Season, SeasonType
from nba_api.stats.endpoints import leaguehustlestatsplayer, leaguegamelog, leaguegamefinder, playbyplayv2, playerestimatedmetrics, leaguedashplayerstats, playerdashboardbyshootingsplits, playerdashboardbygeneralsplits, leaguedashplayerbiostats, leaguedashplayershotlocations
from nba_api.stats.static import players
import pandas as pd
import numpy as np
import time

In [2]:
seasons = ['2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']

# Collecting Box Score Stats

In [ ]:
player_stats = []

for s in seasons:
    season_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=s, season_type_all_star=SeasonType.regular).get_data_frames()[0]
    season_stats['SEASON'] = s
    player_stats.append(season_stats)
    time.sleep(2) 

In [5]:
target_cols1 = ['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'GP', 'MIN', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'TOV', 'PF', 'FGA', 'FG_PCT', 'FG3A', 'FG3_PCT']
per_game_cols = ['PTS', 'AST', 'REB', 'STL', 'BLK', 'TOV', 'PF', 'FGA', 'FG3A']

In [7]:
box_score_stats = [df[target_cols1] for df in player_stats.copy()]
per_game_box = []

for df in box_score_stats:
    temp = df[per_game_cols].div(df['GP'], axis=0)
    temp[target_cols1[:3]] = df[target_cols1[:3]]
    per_game_box.append(temp)


In [ ]:
all_player_stats = pd.concat(per_game_box, ignore_index=True)
all_player_stats.to_csv('../datasets/all_player_stats.csv', index=False)

# Collecting Hustle Stats

In [8]:
hustle_stats = []

In [9]:
for s in seasons:
    season_stats = leaguehustlestatsplayer.LeagueHustleStatsPlayer(season=s).get_data_frames()[0]
    season_stats['SEASON'] = s
    hustle_stats.append(season_stats)
    time.sleep(2) 

In [ ]:
all_hustle_stats = pd.concat(hustle_stats, ignore_index=True)
all_hustle_stats.to_csv('../datasets/all_hustle_stats.csv', index=False)

C:\Users\Fauzan Aryaputra\AppData\Local\Temp\ipykernel_27544\1620728109.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_hustle_stats = pd.concat(hustle_stats, ignore_index=True)


# Collecting Miscellaneous Advanced Stats

In [11]:
advanced_stats = []

In [12]:
for s in seasons:
    season_stats = playerestimatedmetrics.PlayerEstimatedMetrics(season=s).get_data_frames()[0]
    season_stats['SEASON'] = s
    advanced_stats.append(season_stats)
    time.sleep(2) 

In [ ]:
all_advanced_stats = pd.concat(advanced_stats, ignore_index=True)
all_advanced_stats.to_csv('../datasets/all_advanced_stats.csv', index=False)

# Collecting Shot Tendency Data

In [18]:
# Get all player IDs

active_players = []
shooting_data = []
missed_players = []


for df in player_stats:
    active_players.append(df['PLAYER_ID'].tolist())

In [16]:
def pull_shooting_data(pid, s):
    try:
        pull = playerdashboardbyshootingsplits.PlayerDashboardByShootingSplits(player_id=pid, season= s).get_data_frames()[3]
        pull['PLAYER_ID'] = pid
        pull['SEASON'] = s
        return pull
    except:
        return (pid, s)

In [19]:
for s, pids in enumerate(active_players):
    print('Starting season:', seasons[s])
    for pid in pids:
        pull = pull_shooting_data(pid, seasons[s])
        if isinstance(pull, tuple):
            missed_players.append(pull)
        else:
            shooting_data.append(pull)
        time.sleep(2)

time.sleep(1800)

while missed_players:
    for player in missed_players.copy():
        pull = pull_shooting_data(player[0], player[1])
        if isinstance(pull, tuple):
            continue
        else:
            shooting_data.append(pull)
            missed_players.remove(player)

        time.sleep(2)

Starting season: 2014-15
Starting season: 2015-16
Starting season: 2016-17
Starting season: 2017-18
Starting season: 2018-19
Starting season: 2019-20
Starting season: 2020-21
Starting season: 2021-22
Starting season: 2022-23
Starting season: 2023-24


In [24]:
all_shooting_data = pd.concat(shooting_data, ignore_index=True)

In [ ]:
all_shooting_data.to_csv('../datasets/all_shooting_data.csv', index=False)